Goal of this notebook:

Perform feature selection on our dataset.

Strategy:

Iterate over each project and execute the feature selection

In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display

pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
import warnings
import classifier_utils
warnings.filterwarnings("ignore")

In [2]:
non_features_columns = ["chunk_id", "line_start", "line_end", "line_separator", "kind_conflict", "url", "project"]
non_features_columns.extend(["project_user", "project_name", "path", "file_name", "sha", "leftsha", "rightsha", "basesha"])

In [3]:
selected_dataset = pd.read_csv("../../data/SELECTED_LABELLED_DATASET.csv")
projects = list(selected_dataset['project'].unique())

In [4]:
rf = RandomForestClassifier(random_state=99, n_jobs=5, n_estimators=400, max_features=0.3, min_samples_leaf=1)

# Tree-based feature selection

Uses the feature_importances_ attribute from the Random Forest model to select the most important features. It uses the mean of the importances of all features as a threshold.

In [5]:
import importlib
importlib.reload(classifier_utils)
results_tree, attributes_record_tree = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'tree')

In [6]:
results_tree

,project,N,# attr.,# attr. fs,accuracy,accuracy_fs,improvement
0,Ramblurr__Anki-Android,759.0,129.00,33.00,0.742,0.747,0.021
1,apache__directory-server,652.0,96.00,18.00,0.937,0.929,-0.008
2,android__platform_frameworks_base,2460.0,566.00,54.00,0.817,0.815,-0.003
3,freenet__fred,1012.0,134.00,47.00,0.679,0.675,-0.006
4,alexo__wro4j,1368.0,107.00,31.00,0.584,0.582,-0.004
5,apache__lucene-solr,974.0,124.00,42.00,0.646,0.640,-0.010
6,elastic__elasticsearch,NaN,NaN,NaN,NaN,NaN,NaN
7,getrailo__railo,572.0,90.00,31.00,0.711,0.708,-0.005
8,atlasapi__atlas,782.0,124.00,40.00,0.673,0.647,-0.038
9,hibernate__hibernate-orm,716.0,131.00,33.00,0.601,0.583,-0.030


# Recursive feature elimination:

First, the estimator is trained on the initial set of features and the importance of each feature is obtained. Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.
Currently we are discarding 1 feature per step, using a 5-fold-cross-validation to calculate the accuracy on each step.

In [7]:
import importlib
importlib.reload(classifier_utils)
results_recursive, attributes_record_rec = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'recursive')

In [8]:
results_recursive

,project,N,# attr.,# attr. fs,accuracy,accuracy_fs,improvement
0,Ramblurr__Anki-Android,759.0,129.00,50.0,0.742,0.744,0.010
1,apache__directory-server,652.0,96.00,80.0,0.937,0.936,-0.002
2,android__platform_frameworks_base,2460.0,566.00,324.0,0.817,0.820,0.013
3,freenet__fred,1012.0,134.00,97.0,0.679,0.674,-0.007
4,alexo__wro4j,1368.0,107.00,68.0,0.584,0.582,-0.004
5,apache__lucene-solr,974.0,124.00,81.0,0.646,0.649,0.009
6,elastic__elasticsearch,NaN,NaN,NaN,NaN,NaN,NaN
7,getrailo__railo,572.0,90.00,68.0,0.711,0.713,0.006
8,atlasapi__atlas,782.0,124.00,123.0,0.673,0.668,-0.008
9,hibernate__hibernate-orm,716.0,131.00,91.0,0.601,0.594,-0.012


# IGAR
Selects attributes based on the ranking of their information gain.
Information gain measures the ability of a feature to separate the target classes. The greater the information gain, the better its importance for classification tasks.

Information Gain = Entropy(overall) - Entropy(attribute)

The algorithm has an input value 'n' that is used to select the 'n' attributes with the greatest information gain among all attributes. In this notebook we use n = 82, which was the found in the notebook IGAR_tuning.ipynb.

In [9]:
import importlib
importlib.reload(classifier_utils)
results_IGAR, attributes_record_IGAR = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'IGAR')

In [10]:
results_IGAR

,project,N,# attr.,# attr. fs,accuracy,accuracy_fs,improvement
0,Ramblurr__Anki-Android,759.0,129.00,82.0,0.742,0.742,0.000
1,apache__directory-server,652.0,96.00,82.0,0.937,0.940,0.049
2,android__platform_frameworks_base,2460.0,566.00,82.0,0.817,0.815,-0.002
3,freenet__fred,1012.0,134.00,82.0,0.679,0.666,-0.019
4,alexo__wro4j,1368.0,107.00,82.0,0.584,0.583,-0.001
5,apache__lucene-solr,974.0,124.00,82.0,0.646,0.651,0.015
6,elastic__elasticsearch,NaN,NaN,NaN,NaN,NaN,NaN
7,getrailo__railo,572.0,90.00,82.0,0.711,0.715,0.012
8,atlasapi__atlas,782.0,124.00,82.0,0.673,0.667,-0.008
9,hibernate__hibernate-orm,716.0,131.00,82.0,0.601,0.588,-0.021


## Comparison

In [11]:
df_inner = pd.merge(results_tree, results_recursive, on='project', how='inner', suffixes=('_tree', '_rec'))
df_inner_igar = results_IGAR.add_suffix("_IGAR").rename(columns={"project_IGAR": "project"})
df_inner = pd.merge(df_inner, df_inner_igar, on='project', how='inner')
df_inner.to_csv('feature_selection_comparison.csv', index=False)

accuracy_inner = df_inner.filter(regex=("project|accuracy.*")).copy()
accuracy_inner['improvement_tree'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_tree'], x['accuracy_tree']), axis=1)
accuracy_inner['improvement_rec'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_rec'], x['accuracy_rec']), axis=1)
accuracy_inner['improvement_IGAR'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_IGAR'], x['accuracy_IGAR']), axis=1)
accuracy_inner = accuracy_inner.round(3)
accuracy_inner

,project,accuracy_tree,accuracy_fs_tree,accuracy_rec,accuracy_fs_rec,accuracy_IGAR,accuracy_fs_IGAR,improvement_tree,improvement_rec,improvement_IGAR
0,Ramblurr__Anki-Android,0.742,0.747,0.742,0.744,0.742,0.742,0.019,0.008,0.000
1,apache__directory-server,0.937,0.929,0.937,0.936,0.937,0.940,-0.009,-0.001,0.048
2,android__platform_frameworks_base,0.817,0.815,0.817,0.820,0.817,0.815,-0.002,0.016,-0.002
3,freenet__fred,0.679,0.675,0.679,0.674,0.679,0.666,-0.006,-0.007,-0.019
4,alexo__wro4j,0.584,0.582,0.584,0.582,0.584,0.583,-0.003,-0.003,-0.002
5,apache__lucene-solr,0.646,0.640,0.646,0.649,0.646,0.651,-0.009,0.008,0.014
6,elastic__elasticsearch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,getrailo__railo,0.711,0.708,0.711,0.713,0.711,0.715,-0.004,0.007,0.014
8,atlasapi__atlas,0.673,0.647,0.673,0.668,0.673,0.667,-0.039,-0.007,-0.009
9,hibernate__hibernate-orm,0.601,0.583,0.601,0.594,0.601,0.588,-0.030,-0.012,-0.022


In [12]:
pd.DataFrame(attributes_record_rec, columns=['project', 'attribute', 'information_gain', 'method']).to_csv('attributes_record_rec.csv', index=False)

In [13]:
attributes_record = []
attributes_record.extend(attributes_record_tree)
attributes_record.extend(attributes_record_rec)
attributes_record.extend(attributes_record_IGAR)
attributes_record_df = pd.DataFrame(attributes_record, columns=['project', 'attribute', 'information_gain', 'method'])
attributes_record_df.to_csv('attributes_record.csv', index=False)

In [14]:
attributes_record = pd.read_csv('attributes_record.csv')

## Ranking of features selected by tree method

Counts in how many projects the respective feature was selected using the tree method.

In [15]:
ranking_tree = classifier_utils.get_attribute_selection_ranking(attributes_record, 'tree')
ranking_tree.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)

,attribute,count_selected,average_information_gain,average_ranking
17,chunkRelSize,19.0,1.396814,1.000000
19,fileSize,19.0,1.090939,3.526316
25,Merge isolation time,15.0,1.006587,3.266667
24,Branching time,15.0,0.933865,3.800000
3,right_lines_removed,17.0,0.876816,4.823529
2,right_lines_added,17.0,0.869158,5.823529
1,left_lines_removed,15.0,0.847372,7.066667
0,left_lines_added,16.0,0.817331,8.125000
29,Changed files 1,16.0,0.785025,10.125000
27,Commits 1,15.0,0.743192,10.000000


## Ranking of features selected by recursive method

Counts in how many projects the respective feature was selected using the recursive method.

In [16]:
ranking_recursive = classifier_utils.get_attribute_selection_ranking(attributes_record, 'recursive')
ranking_recursive.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)

,attribute,count_selected,average_information_gain,average_ranking
18,chunkRelSize,19.0,1.418194,1.894737
20,fileSize,18.0,1.172272,4.277778
26,Merge isolation time,17.0,0.929518,3.705882
25,Branching time,17.0,0.928754,3.823529
1,left_lines_removed,18.0,0.841380,6.611111
3,right_lines_removed,18.0,0.839538,8.166667
2,right_lines_added,18.0,0.832305,9.111111
0,left_lines_added,18.0,0.790354,8.277778
32,Changed files 1,18.0,0.756030,9.944444
16,fileCC,18.0,0.752752,10.000000


## Ranking of features selected by IGAR method

Counts in how many projects the respective feature was selected using the IGAR method. 

The information gain column is an average among all projects.

In [17]:
import importlib
importlib.reload(classifier_utils)
ranking_IGAR = classifier_utils.get_attribute_selection_ranking(attributes_record, 'IGAR')
ranking_IGAR.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)

,attribute,count_selected,average_information_gain,average_ranking
27,chunkRelSize,15.0,1.303072,2.133333
12,fileSize,15.0,1.004468,4.733333
3,Merge isolation time,19.0,0.780967,3.894737
1,Branching time,19.0,0.780469,3.947368
2,right_lines_added,20.0,0.760997,8.350000
7,left_lines_removed,19.0,0.742402,6.421053
9,right_lines_removed,19.0,0.735100,7.894737
0,left_lines_added,20.0,0.722940,7.800000
14,Changed files 1,19.0,0.712724,8.894737
10,fileCC,15.0,0.665897,10.066667
